## Analyzing dome monitor with a labJack T7
Uses 2 analog input (AINs) to read the data.

Craig Lage - Jul 5, 2023

In [ ]:
import sys, os
import time, datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
dir = "/scratch/labJackData/"

files = os.listdir(dir)
counter = 0
for file in files:
    if file.split("_")[0] == "Dome":
        filename = dir+file
        
        pkl_file = open(filename, 'rb')
        df = pkl.load(pkl_file)
        pkl_file.close()
        print(filename, len(df), df.columns)
        if counter == 0:
            merged_df = df
        else:
            merged_df = pd.concat([merged_df, df], axis=1)
        counter += 1


In [ ]:
len(merged_df)

In [ ]:
client = EfdClient('summit_efd')
start = Time("2023-07-05 22:31:30Z", scale='utc')
end = Time("2023-07-06 01:02:00Z", scale='utc')
state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)


In [ ]:
fig, ax = plt.subplots(1,1)
ax2 = ax.twinx()
state['state'].plot(ax=ax2, color='blue', label='Actual State')
ax2.set_ylim(0,6)
ax2.set_yticks([1,2,3,4,5])
ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])


merged_df["AIN0"].plot(ax=ax, label="Monitor")
ax.set_ylim(-0.5,10.0)
start_plot = Time("2023-07-06 00:40:00Z", scale='utc')
end_plot = Time("2023-07-06 00:52:00Z", scale='utc')

ax.set_xlim(start_plot.isot, end_plot.isot)
ax.get_legend().remove()
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_05Jul23.png")